<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/semantic_vector2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=9044484851e2db030a1c6aa1251149af29c648cb89abb46f88c62f65bddc580a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)


In [5]:
from sentence_transformers import SentenceTransformer, util

query_embedding = model.encode('How big is London')
passage_embedding = model.encode(['London has 9,787,426 inhabitants at the 2011 census',
                                  'London is known for its finacial district'])

print("Similarity:", util.dot_score(query_embedding, passage_embedding))

Similarity: tensor([[0.5627, 0.5645]])


In [7]:
query_embedding = model.encode('i am good person')
passage_embedding = model.encode(['i am beautiful person',
                                  'i am bad good'])
print("Similarity:", util.dot_score(query_embedding, passage_embedding))

Similarity: tensor([[0.5520, 0.6742]])


In [8]:
query_embedding = model.encode('double-hummer alignment exceptions')
passage_embedding = model.encode(['floating point alignment exceptions',
                                  '	<*> torus receiver x+ input pipe error(s) (dcr <*>) detected and corrected'])
print("Similarity:", util.dot_score(query_embedding, passage_embedding))

Similarity: tensor([[0.5786, 0.2495]])


In [20]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/HDFS_templates.csv')
df.head()

,EventId,EventTemplate
0,E1,<*>Adding an already existing block<*>
1,E2,<*>Verification succeeded for<*>
2,E3,<*>Served block<*>to<*>
3,E4,<*>Got exception while serving<*>to<*>
4,E5,<*>Receiving block<*>src:<*>dest:<*>


In [21]:
df['EventTemplate'][0]

'<*>Adding an already existing block<*>'

In [22]:
len(df['EventTemplate'])

30

In [23]:
list(df['EventTemplate'][1:])

['<*>Verification succeeded for<*>',
 '<*>Served block<*>to<*>',
 '<*>Got exception while serving<*>to<*>',
 '<*>Receiving block<*>src:<*>dest:<*>',
 '<*>Received block<*>src:<*>dest:<*>of size<*>',
 '<*>writeBlock<*>received exception<*>',
 '<*>PacketResponder<*>for block<*>Interrupted<*>',
 '<*>Received block<*>of size<*>from<*>',
 '<*>PacketResponder<*>Exception<*>',
 '<*>PacketResponder <*> for block <*> terminating<*>',
 '<*>:Exception writing block<*>to mirror<*>',
 '<*>Receiving empty packet for block<*>',
 '<*>Exception in receiveBlock for block<*>',
 '<*>Changing block file offset of block<*>from<*>to<*>meta file offset to<*>',
 '<*>:Transmitted block<*>to<*>',
 '<*>:Failed to transfer<*>to<*>got<*>',
 '<*>Starting thread to transfer block<*>to<*>',
 '<*>Reopen Block<*>',
 '<*>Unexpected error trying to delete block<*>BlockInfo not found in volumeMap<*>',
 '<*>Deleting block<*>file<*>',
 '<*>BLOCK* NameSystem<*>allocateBlock:<*>',
 '<*>BLOCK* NameSystem<*>delete:<*>is added to

In [26]:
query_embedding = model.encode(df['EventTemplate'][0])
passage_embedding = model.encode(list(df['EventTemplate']))
similarity = util.dot_score(query_embedding, passage_embedding)
print("Similarity:", similarity)

Similarity: tensor([[1.0000, 0.3306, 0.5373, 0.2941, 0.4918, 0.4210, 0.4895, 0.3000, 0.3977,
         0.0989, 0.2919, 0.4331, 0.4358, 0.4876, 0.5128, 0.4233, 0.1819, 0.5224,
         0.5560, 0.4074, 0.5925, 0.5645, 0.5608, 0.6515, 0.5734, 0.4526, 0.4146,
         0.3894, 0.4101, 0.5846]])


In [34]:
import torch
torch.topk(similarity,5)[]

torch.return_types.topk(
values=tensor([[1.0000, 0.6515, 0.5925, 0.5846, 0.5734]]),
indices=tensor([[ 0, 23, 20, 29, 24]]))

In [43]:
torch. squeeze(similarity)

tensor([1.0000, 0.3306, 0.5373, 0.2941, 0.4918, 0.4210, 0.4895, 0.3000, 0.3977,
        0.0989, 0.2919, 0.4331, 0.4358, 0.4876, 0.5128, 0.4233, 0.1819, 0.5224,
        0.5560, 0.4074, 0.5925, 0.5645, 0.5608, 0.6515, 0.5734, 0.4526, 0.4146,
        0.3894, 0.4101, 0.5846])

In [46]:
size = len(df['EventTemplate'])
tensor = torch.zeros([size, size], dtype=torch.float64)
for index,template in enumerate(df['EventTemplate']):
  query_embedding = model.encode(template)
  passage_embedding = model.encode(list(df['EventTemplate']))
  similarity = util.dot_score(query_embedding, passage_embedding)
  tensor[index] = torch. squeeze(similarity)


In [53]:
torch.min(tensor)

tensor(0.0989, dtype=torch.float64)

In [57]:
torch.min(tensor, dim=0, keepdim=False)

torch.return_types.min(
values=tensor([0.0989, 0.2040, 0.2546, 0.2282, 0.3197, 0.2950, 0.3909, 0.2170, 0.2776,
        0.0989, 0.2354, 0.2904, 0.2895, 0.3736, 0.1052, 0.2870, 0.1669, 0.3284,
        0.2562, 0.1815, 0.1612, 0.1541, 0.2273, 0.1681, 0.2124, 0.1884, 0.2186,
        0.2093, 0.1990, 0.1708], dtype=torch.float64),
indices=tensor([ 9, 25, 16, 14, 26,  3, 14, 19,  3,  0, 16, 19, 14, 14,  9, 27, 14,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 14,  9]))

In [58]:
tensor[0][9]

tensor(0.0989, dtype=torch.float64)

In [59]:
df['EventTemplate'][9]

'<*>PacketResponder<*>Exception<*>'

In [68]:
torch.topk(tensor,30)

torch.return_types.topk(
values=tensor([[1.0000, 0.6515, 0.5925, 0.5846, 0.5734, 0.5645, 0.5608, 0.5560, 0.5373,
         0.5224, 0.5128, 0.4918, 0.4895, 0.4876, 0.4526, 0.4358, 0.4331, 0.4233,
         0.4210, 0.4146, 0.4101, 0.4074, 0.3977, 0.3894, 0.3306, 0.3000, 0.2941,
         0.2919, 0.1819, 0.0989],
        [1.0000, 0.5633, 0.5120, 0.5081, 0.4705, 0.4682, 0.4678, 0.4672, 0.4391,
         0.4223, 0.3998, 0.3846, 0.3844, 0.3825, 0.3724, 0.3658, 0.3558, 0.3443,
         0.3306, 0.3290, 0.2820, 0.2799, 0.2719, 0.2688, 0.2541, 0.2505, 0.2457,
         0.2126, 0.2073, 0.2040],
        [1.0000, 0.6575, 0.6438, 0.6413, 0.6354, 0.6027, 0.5911, 0.5753, 0.5744,
         0.5580, 0.5576, 0.5407, 0.5373, 0.5321, 0.5308, 0.5186, 0.5168, 0.5103,
         0.5065, 0.4979, 0.4915, 0.4595, 0.4141, 0.4084, 0.3827, 0.3825, 0.3783,
         0.3632, 0.2968, 0.2546],
        [1.0000, 0.6384, 0.6354, 0.5885, 0.5797, 0.5417, 0.5081, 0.4861, 0.4682,
         0.4498, 0.4495, 0.4429, 0.4414, 0.4183, 0.3956,

In [71]:
torch.sum(tensor, dim=0)

tensor([13.8069, 11.4445, 15.5598, 12.7691, 15.9980, 14.3309, 15.6007, 13.2351,
        14.3737, 10.5456, 13.1550, 13.9947, 14.5591, 16.2084, 11.7023, 14.4359,
        10.4469, 15.1565, 13.9656, 12.1547, 15.1126, 13.8813, 15.3937, 14.6299,
        14.3318, 12.9265, 12.5149, 12.3938, 12.0844, 15.3285],
       dtype=torch.float64)

In [73]:
# least and max similarity templates with others
print(df['EventTemplate'][16])
print(df['EventTemplate'][13])


<*>:Failed to transfer<*>to<*>got<*>
<*>Exception in receiveBlock for block<*>


In [70]:
# max similarity
print(df['EventTemplate'][5])
print(df['EventTemplate'][4])

<*>Received block<*>src:<*>dest:<*>of size<*>
<*>Receiving block<*>src:<*>dest:<*>


In [67]:

print(df['EventTemplate'][28])
print(df['EventTemplate'][13])

<*>PendingReplicationMonitor timed out block<*>
<*>Exception in receiveBlock for block<*>
